In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN9001_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval_activity as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp1.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((1, 700165, 77), (1, 700165, 18))

In [7]:
test_data.shape, test_gt.shape

((1, 120516, 77), (1, 120516, 18))

## Model

In [10]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 1

convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu
n_classes = 18

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[0,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


In [11]:
pos_weight.shape

(18,)

In [ ]:
pos_weight

In [13]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [14]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [15]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [16]:
model.build_conv_net(pos_weight_values = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [19]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'activity')

In [ ]:
trainer.train(max_iter = 20000,
                train_eval_step = 1, 
                validation_eval_step = 1,
                display_step = 1)

Train Batch Evaluation
AUC = 0.080 vs. 0.076
F1 = 0.123 vs. 0.107
Accuracy = 0.007 vs. 0.056
Validation
AUC = 0.070 vs. 0.070
F1 = 0.102 vs. 0.100
Accuracy = 0.007 vs. 0.056
Minibatch Loss= 1.455329
Train Batch Evaluation
AUC = 0.084 vs. 0.077
F1 = 0.131 vs. 0.110
Accuracy = 0.008 vs. 0.056
Validation
AUC = 0.077 vs. 0.073
F1 = 0.115 vs. 0.115
Accuracy = 0.007 vs. 0.056
Minibatch Loss= 1.325861
Train Batch Evaluation
AUC = 0.092 vs. 0.077
F1 = 0.143 vs. 0.108
Accuracy = 0.011 vs. 0.056
Validation
AUC = 0.086 vs. 0.070
F1 = 0.134 vs. 0.099
Accuracy = 0.009 vs. 0.056
Minibatch Loss= 1.265858
Train Batch Evaluation
AUC = 0.101 vs. 0.076
F1 = 0.159 vs. 0.108
Accuracy = 0.015 vs. 0.056
Validation
AUC = 0.093 vs. 0.069
F1 = 0.144 vs. 0.102
Accuracy = 0.016 vs. 0.056
Minibatch Loss= 1.240830
Train Batch Evaluation
AUC = 0.112 vs. 0.076
F1 = 0.178 vs. 0.108
Accuracy = 0.025 vs. 0.056
Validation
AUC = 0.100 vs. 0.069
F1 = 0.157 vs. 0.099
Accuracy = 0.021 vs. 0.056
Minibatch Loss= 1.226547
Train

In [22]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/CNN9001_opp/CNN9001_opp-4000'

In [ ]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [23]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.537 vs. 0.069
F1 = 0.586 vs. 0.103
Accuracy = 0.919 vs. 0.056


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [21]:
1

1